In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import itertools
import seaborn as sns
import pickle
import joblib
import os

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

# Functions

In [2]:
def save_model(model, model_path):
    with open(model_path, 'wb') as f:
        pickle.dump(model, f)

def load_model(model_path):
    with open(model_path, 'rb') as f:
        return pickle.load(f)

def load_dataset(folder_name):
    with open(f'../data/{folder_name}/train.pkl', 'rb') as f:
        X_train, y_train = pickle.load(f)
    with open(f'../data/{folder_name}/val.pkl', 'rb') as f:
        X_val, y_val = pickle.load(f)
    with open(f'../data/{folder_name}/test.pkl', 'rb') as f:
        X_test, y_test = pickle.load(f)
    
    scaler_y = None
    if folder_name in ['norm', 'hot_norm']:
        with open(f'../data/scaler_y.pkl', 'rb') as f:
            scaler_y = pickle.load(f)
    
    return X_train, X_val, X_test, y_train, y_val, y_test, scaler_y

def evaluate_model(model, model_path, X_train, X_val, X_test, y_train, y_val, y_test, scaler_y=None, dataset_name=''):
    performances = []
    data_splits = [(X_train, y_train, 'train'), (X_val, y_val, 'val'), (X_test, y_test, 'test')]
    
    for x, y, nsplit in data_splits:
        preds = model.predict(x)
        r2 = r2_score(y, preds)
        rmse = np.sqrt(mean_squared_error(y, preds))
        mae = mean_absolute_error(y, preds)
        
        # Update model name to include number of neighbors, weights, and metric if KNN
        model_name = model.__class__.__name__
        if model_name == 'KNeighborsRegressor':
            model_name += f" (k={model.n_neighbors}, weights={model.weights}, metric={model.metric})"
        
        performances.append({
            'model': model_name,
            'dataset': dataset_name,
            'split': nsplit,
            'rmse': rmse.round(4),
            'mae': mae.round(4),
            'r2': r2.round(4),
            'inversed': False,
            'model_path': model_path
        })
        
        if scaler_y is not None:
            preds_original = scaler_y.inverse_transform(preds.reshape(-1, 1)).flatten()
            y_original = scaler_y.inverse_transform(y.reshape(-1, 1)).flatten()
            
            r2_original = r2_score(y_original, preds_original)
            rmse_original = np.sqrt(mean_squared_error(y_original, preds_original))
            mae_original = mean_absolute_error(y_original, preds_original)
            
            performances.append({
                'model': model_name,
                'dataset': dataset_name,
                'split': nsplit,
                'rmse': rmse_original.round(4),
                'mae': mae_original.round(4),
                'r2': r2_original.round(4),
                'inversed': True,
                'model_path': model_path
            })
    
    return performances

# Custom grid search function
def custom_grid_search_knn(X_train, y_train, param_grid, dataset_name):
    model_dir = '../models'
    os.makedirs(model_dir, exist_ok=True)
    
    for params in param_grid:
        knn = KNeighborsRegressor(**params)
        knn.fit(X_train, y_train)
        
        model_path = os.path.join(model_dir, f'knn_{params["n_neighbors"]}_neighbors_{params["weights"]}_{params["metric"]}_{dataset_name}.pkl')
        save_model(knn, model_path)
        print(f"Trained and saved KNeighborsRegressor model with parameters {params} to {model_path}")


# Fit models

# KNN

In [3]:
# Define parameter grid for KNN
param_grid_knn = [{'n_neighbors': k, 'weights': w, 'metric': m} for k in [5, 15, 20, 25, 30, 50] for w in ['uniform', 'distance'] for m in ['euclidean', 'manhattan']]

# Initialize a list to store all performances
all_performances = []

# List of dataset names and their corresponding folder names
dataset_names = ['hot_norm']

# Load each dataset and perform custom grid search
for dataset_name in dataset_names:
    X_train, X_val, X_test, y_train, y_val, y_test, scaler_y = load_dataset(dataset_name)
    custom_grid_search_knn(X_train, y_train, param_grid_knn, dataset_name)
    
    # Evaluate all saved models
    for params in param_grid_knn:
        model_path = os.path.join('../models', f'knn_{params["n_neighbors"]}_neighbors_{params["weights"]}_{params["metric"]}_{dataset_name}.pkl')
        knn = load_model(model_path)
        performances = evaluate_model(knn, model_path, X_train, X_val, X_test, y_train, y_val, y_test, scaler_y=scaler_y, dataset_name=dataset_name)
        all_performances.extend(performances)

# Display performances
for performance in all_performances:
    print(performance)

# Save performances to a CSV file
perf_df = pd.DataFrame(all_performances)
perf_df.to_csv('../data/knn_performances.csv', index=False)


Trained and saved KNeighborsRegressor model with parameters {'n_neighbors': 5, 'weights': 'uniform', 'metric': 'euclidean'} to ../models/knn_5_neighbors_uniform_euclidean_hot_norm.pkl
Trained and saved KNeighborsRegressor model with parameters {'n_neighbors': 5, 'weights': 'uniform', 'metric': 'manhattan'} to ../models/knn_5_neighbors_uniform_manhattan_hot_norm.pkl
Trained and saved KNeighborsRegressor model with parameters {'n_neighbors': 5, 'weights': 'distance', 'metric': 'euclidean'} to ../models/knn_5_neighbors_distance_euclidean_hot_norm.pkl
Trained and saved KNeighborsRegressor model with parameters {'n_neighbors': 5, 'weights': 'distance', 'metric': 'manhattan'} to ../models/knn_5_neighbors_distance_manhattan_hot_norm.pkl
Trained and saved KNeighborsRegressor model with parameters {'n_neighbors': 15, 'weights': 'uniform', 'metric': 'euclidean'} to ../models/knn_15_neighbors_uniform_euclidean_hot_norm.pkl
Trained and saved KNeighborsRegressor model with parameters {'n_neighbors'